# Initial envirnoment



In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pylab as plt
import tensorflow_hub as hub
from tensorflow.keras import layers
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.applications.inception_v3 import InceptionV3

try:
  use_cuda = True
except:
  pass  


#test GPU
x = tf.random_uniform([3, 3])

print("Is there a GPU available: "),
print(tf.test.is_gpu_available())

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith(''))


## google drive
from google.colab import drive
drive.mount('/content/drive')

Is there a GPU available: 
False
Is the Tensor on GPU #0:  
True
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Param

In [0]:
model_name = "GoogleNet Model None weight"
epochs = 30
data_path_floder ="my dataset"





#image parameters
IMAGE_SHAPE = (299, 299)
validation_split = 0.15
batch_size = 32
classes = 24




In [3]:

'''
建立存储目录
'''
def mkdir(path):
	folder = os.path.exists(path)
	if not folder:                   
		os.makedirs(path)            
		print ("---  new folder...  ---")
		print ("---  OK  ---")
	else:
		print ("---  There is this folder!  ---")

data_path = "/content/drive/My Drive/datasets/" + data_path_floder +"/"
save_path = "/content/drive/My Drive/model_saving/"+data_path_floder+"/"+model_name+"/epochs="+str(epochs)
mkdir(save_path)


f = open(save_path+"/info.txt", 'w')
f.write("dataset: "+ data_path_floder+"\n"
        "model name: "+ model_name+"\n"
        "training epochs: "+str(epochs)+"\n" )
f.close()


print(data_path)
save_path
path_5h = save_path+"/"+ model_name+"_"+data_path_floder+"_epochs = "+str(epochs)+".h5"
print(path_5h)

---  There is this folder!  ---
/content/drive/My Drive/datasets/my dataset/
/content/drive/My Drive/model_saving/my dataset/GoogleNet Model None weight/epochs=30/GoogleNet Model None weight_my dataset_epochs = 30.h5


# Dealing Data 


## Dataset Generator

In [4]:

#dataset generator
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,
                                                                  validation_split=validation_split)

#traing set
train_data = image_generator.flow_from_directory(str(data_path),
                                                 target_size=IMAGE_SHAPE,
                                                 batch_size=batch_size,
                                                 subset = 'training'
                                                )
#testing set
test_data = image_generator.flow_from_directory(str(data_path),
                                                 target_size=IMAGE_SHAPE,
                                                 batch_size=batch_size,
                                                 subset = 'validation'
                                                )



print("traing data shape: ",train_data.image_shape)
print("training data batch size: ",train_data.batch_size)
print("trainig data classes number: ",train_data.num_classes)
print("training data samples number: ",train_data.samples,"\n","\n")

print("test data shape: ",test_data.image_shape)
print("test data batch size: ",test_data.batch_size)
print("test data classes number: ",test_data.num_classes)
print("test data sample number: ",test_data.samples)


Found 17196 images belonging to 24 classes.
Found 3026 images belonging to 24 classes.
traing data shape:  (299, 299, 3)
training data batch size:  32
trainig data classes number:  24
training data samples number:  17196 
 

test data shape:  (299, 299, 3)
test data batch size:  32
test data classes number:  24
test data sample number:  3026


# Building GoogleNet

In [5]:

base_model = InceptionV3( weights = None, include_top=False, input_tensor=Input(shape=(299, 299, 3)))
x = base_model.output
x = AveragePooling2D(pool_size=(8, 8))(x)
x = Dropout(.4)(x)
x = Flatten()(x)
x = Dense(1000,activation='relu')(x)
x = Dropout(.4)(x)
predictions = Dense(train_data.num_classes,activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
#model.summary()



#compile
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss='categorical_crossentropy',
  metrics=['acc'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
from tensorflow import keras
keras.utils.plot_model(model, 'multi_input_and_output_model.png', show_shapes=True)

# Fit

In [0]:
#for plot step acc and loss(not bitch just step)
class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []

  def on_train_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
    self.model.reset_metrics()


#fit
steps_per_epoch = np.ceil(train_data.samples/train_data.batch_size)
batch_stats_callback = CollectBatchStats()
history = model.fit(train_data, 
                    epochs=epochs,
                    validation_data = test_data,
                    callbacks = [batch_stats_callback])

Epoch 1/30
523/538 [============================>.] - ETA: 7:56 - loss: 2.1797 - acc: 0.4062

#plot 

## plot step acc and loss

In [0]:
# plot param
title_acc = "Step Accuracy of "+model_name
title_loss = "Step Loss function of "+model_name
figure_path = save_path+"/figures"
mkdir(figure_path)


plt.rcParams['savefig.dpi'] = 200 
plt.rcParams['figure.dpi'] = 200 

plt.figure()
plt.ylabel("Accuracy")
plt.xlabel("Training Steps")  
plt.title(title_acc) 
y_ticks = np.arange(0, 1, 0.05)
plt.yticks(y_ticks)          #显示范围为-2至2，每0.2显示一刻度
plt.grid(axis="y")
plt.plot(batch_stats_callback.batch_acc)
fig = plt.gcf()
fig.savefig(figure_path+"/"+title_acc+".png")
plt.show()


plt.figure()
plt.ylabel("Loss")
plt.xlabel("Training Steps")
plt.title(title_loss) 
y_ticks = np.arange(0, 5, 0.25)
plt.yticks(y_ticks)          #显示范围为-2至2，每0.2显示一刻度
plt.grid(axis="y")
plt.plot(batch_stats_callback.batch_losses)
fig = plt.gcf()
fig.savefig(figure_path+"/"+title_loss+".png")
plt.show()


## plot epoch acc loss and valid acc and valid loss

In [0]:
epoch_acc = "Epochs Accuracy of "+model_name
epoch_loss = "Epochs Loss function of "+model_name


# acc
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title(epoch_acc )
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
fig = plt.gcf()
fig.savefig(figure_path+"/"+epoch_acc+".png")
plt.show()

plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title(epoch_loss)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
fig = plt.gcf()
fig.savefig(figure_path+"/"+epoch_loss+".png")
plt.show()

# Evaluation

In [0]:
# evaluat model by test data
loss, acc = model.evaluate(test_data)

# Prediction 

## Plot samples

In [0]:
# pick up a bitch test set
image_test, label_test = test_data.next()


#decode class name from index number 
class_names = sorted(test_data.class_indices.items(), key=lambda pair:pair[1])
class_names = np.array([key.title() for key, value in class_names])


predicted_batch = model.predict(image_test)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]
label_id = np.argmax(label_test, axis=-1)


##plot and print out the acc 
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_test[n])
  color = "green" if predicted_id[n] == label_id[n] else "red"
  plt.title(predicted_label_batch[n].title(), color=color)
  plt.axis('off')
  plt.suptitle(model_name+" Model predictions \n(green: correct, red: incorrect)")

fig = plt.gcf()
fig.savefig(figure_path+"/"+"demo.png")
plt.show()

loss, acc = model.evaluate(image_test, label_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## Confusion Matrix

In [0]:
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import itertools


test_data = image_generator.flow_from_directory(str(data_path),
                                                 target_size=IMAGE_SHAPE,
                                                 batch_size=batch_size,
                                                 subset = 'validation',
                                                shuffle = False
                                                )

class_names = sorted(test_data.class_indices.items(), key=lambda pair:pair[1])
class_names = np.array([key.title() for key, value in class_names])




plt.rcParams['savefig.dpi'] = 300 
plt.rcParams['figure.dpi'] = 300 

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues,
                          class_ = 8):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if (class_ == 24):

      size = 4
    else:
      size = 10




    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90,size = size)
    plt.yticks(tick_marks, classes,size = size)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black",
                 size = size)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')





pred = model.predict(test_data)
pred_index = np.argmax(pred, axis=-1)
cm=tf.math.confusion_matrix(test_data.classes,pred_index)

##convert tensor to np.int32
sess = tf.Session()
with sess.as_default():
  cm_numpy=(cm.eval())

cm_numpy = cm_numpy.astype(np.int16)

##Google lab can not plt it just print out and plot by my own laptop
with tf.Session() as sess:  print(cm.eval()) 
print(class_names)

plot_confusion_matrix(cm_numpy,class_names,class_= train_data.num_classes)
fig = plt.gcf()
fig.savefig(figure_path+"/confusion matrix.png")
plt.show()


# Model saving

## saving h5 model

In [0]:
model.save(path_5h)

### Reload h5

In [0]:
model_h5 = tf.keras.models.load_model(path_5h)

## Saving keras model


In [0]:
tf.keras.experimental.export_saved_model(model, save_path+"/model_save")
# reloaded_model = tf.keras.experimental.load_from_saved_model(model,save_path)

In [0]:
reloaded_model = tf.keras.experimental.load_from_saved_model(save_path+"/model_save")

In [0]:
reloaded_model.summary()